In [7]:
import torch
import torch.nn as nn
from torch.autograd import Variable

torch.manual_seed(777)
idx2char = ['h', 'i', 'e', 'l', 'o']
# Teach hihell -> ihello
x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
y_data = [1, 0, 2, 3, 3, 4]    # ihello

In [8]:
inputs = Variable(torch.LongTensor(x_data))
labels = Variable(torch.LongTensor(y_data))

num_classes = 5
input_size = 5
embedding_size = 10  # embedding size
hidden_size = 5  # output from the LSTM.
batch_size = 1   # one sentence
sequence_length = 6  # |ihello| == 6
num_layers = 1  

In [9]:
class Model(nn.Module):
    def __init__(self, num_layers, hidden_size):
        super(Model, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.rnn = nn.RNN(input_size=embedding_size,
                          hidden_size=5, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))

        emb = self.embedding(x)
        emb = emb.view(batch_size, sequence_length, -1)
        out, _ = self.rnn(emb, h_0)
        return self.fc(out.view(-1, num_classes))


In [10]:
model = Model(num_layers, hidden_size)
print(model)

Model(
  (embedding): Embedding(5, 10)
  (rnn): RNN(10, 5, batch_first=True)
  (fc): Linear(in_features=5, out_features=5, bias=True)
)


In [11]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

In [13]:
for epoch in range(100):
    outputs = model(inputs)
    optimizer.zero_grad()
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    _, idx = outputs.max(1)
    idx = idx.data.numpy()
    result_str = [idx2char[c] for c in idx.squeeze()]
    print("epoch: %d, loss: %1.3f" % (epoch + 1, loss.item()))
    print("Predicted string: ", ''.join(result_str)) # transform from list to string

print("Learning finished!")

epoch: 1, loss: 1.396
Predicted string:  oheiii
epoch: 2, loss: 1.132
Predicted string:  iheloo
epoch: 3, loss: 0.949
Predicted string:  ihello
epoch: 4, loss: 0.798
Predicted string:  ihelll
epoch: 5, loss: 0.659
Predicted string:  ihelll
epoch: 6, loss: 0.547
Predicted string:  ihelll
epoch: 7, loss: 0.449
Predicted string:  ihelll
epoch: 8, loss: 0.384
Predicted string:  ihelll
epoch: 9, loss: 0.343
Predicted string:  ihello
epoch: 10, loss: 0.304
Predicted string:  ihello
epoch: 11, loss: 0.235
Predicted string:  ihello
epoch: 12, loss: 0.227
Predicted string:  ihello
epoch: 13, loss: 0.239
Predicted string:  iheloo
epoch: 14, loss: 0.233
Predicted string:  iheloo
epoch: 15, loss: 0.146
Predicted string:  ihello
epoch: 16, loss: 0.320
Predicted string:  ihelll
epoch: 17, loss: 0.139
Predicted string:  ihello
epoch: 18, loss: 0.228
Predicted string:  iheloo
epoch: 19, loss: 0.242
Predicted string:  iheloo
epoch: 20, loss: 0.191
Predicted string:  iheloo
epoch: 21, loss: 0.115
Predic